In [ ]:
df_train1 = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
cat_mtpl = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}

for category in cat_mtpl:
    df_train1[category] = df_train1[category] * cat_mtpl[category]

df_train1['score'] = df_train1.loc[:, 'toxic':'identity_hate'].mean(axis=1)

df_train1['y'] = df_train1['score']

min_len = (df_train1['y'] > 0).sum()  # len of toxic comments
df_y0_undersample = df_train1[df_train1['y'] == 0].sample(n=min_len*2, random_state=41)  # take non toxic comments
df_train1 = pd.concat([df_train1[df_train1['y'] > 0], df_y0_undersample])  # make new df
df_train1 = df_train1[['comment_text','y']].rename(columns={'comment_text':'text'})

import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

df_train1['text'] = df_train1['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df_train1["text"] = df_train1["text"].apply(lambda text: lemmatize_words(text))

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

raw_tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
raw_tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)
raw_tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

from datasets import Dataset

dataset = Dataset.from_pandas(df_train1[['text']])

def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]
        
raw_tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

def dummy_fun(doc):
    return doc
def vec_train_valition_preds(df):
    labels = df['y']
    comments = df['text']
    comments = [text.lower() for text in comments]
    tokenized_comments = tokenizer(comments)['input_ids']
    
    vectorizer = TfidfVectorizer(
    analyzer = 'word',
    tokenizer = dummy_fun,
    preprocessor = dummy_fun,
    token_pattern = None)
    comments_tr = vectorizer.fit_transform(tokenized_comments)
    
    model = Ridge(random_state=2021)
    model.fit(comments_tr,labels)

    less_toxic_comments = df_valid['less_toxic']
    more_toxic_comments = df_valid['more_toxic']
    
    less_toxic_comments = tokenizer(less_toxic_comments.to_list())['input_ids']
    more_toxic_comments = tokenizer(more_toxic_comments.to_list())['input_ids']
    
    less_toxic = vectorizer.transform(less_toxic_comments)
    more_toxic = vectorizer.transform(more_toxic_comments)
    
    # make predictions
    y_pred_less = model.predict(less_toxic)
    y_pred_more = model.predict(more_toxic)

    print((y_pred_less < y_pred_more).mean())
    
    texts = df_test['text']
    texts = tokenizer(texts.to_list())['input_ids']
    texts = vectorizer.transform(texts)
    p = model.predict(texts)
    return p

p = vec_train_valition_preds(df_train1)